In [1]:
#Dependencies
import numpy as np
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from nltk.tokenize import word_tokenize
import seaborn as sns
import matplotlib.pyplot as plt

/Users/pw/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Data Upload function
def csv_upload(path):
    data = pd.read_csv(path)
    return data

In [3]:
path = '/Users/pw/Desktop/ZindiMl/Project 2/vendors.csv'
vendors = csv_upload(path)

In [4]:
new_data = vendors[['id','vendor_rating','vendor_tag_name']]

In [5]:
#Recommend Function
#data_sample = []
def recommend_func(data_sample,id):
    
    #global data_sample
    global cosine_sim
    global sim_scores
    global tfidf_matrix
    global corpus_index
    global feature
    global rest_indices
    global idx
    
    #data_sample = new_data.loc[new_data['id']==id]
    data_sample.reset_index(level=0,inplace=True)
    
    #Feature Extraction
    data_sample['Split'] = 'X'
    
    for i in range(0,data_sample.index[-1]):
        split_data = re.split(r'[,]',str(data_sample['vendor_tag_name'][i]))

        for k,l in enumerate(split_data):
            split_data[k] = (split_data[k].replace(" ",""))
        split_data = ' '.join(split_data[:])
        data_sample['Split'].iloc[i] = split_data
    
    #print(data_sample)
    #TF-IDF vectorizer
    #Extracting Stopword
    
    tfidf = TfidfVectorizer(stop_words = 'english')
    
    #Replace NaN for empty string
    data_sample['Split'] = data_sample['Split'].fillna('')
    
    #Applying TF-IDF Vectorizer
    tfidf_matrix = tfidf.fit_transform(data_sample['Split'])
    
    
    #Feature for cosine similarities
    feature = tfidf.get_feature_names()
    
    #Compute the cosine similarity matrix
    cosine_sim = linear_kernel(tfidf_matrix,tfidf_matrix)
    
    # Column names are using for index
    corpus_index = [n for n in data_sample['Split']]
    
    # Construct a reverse map of indices
    
    indices = pd.Series(data_sample.index, index = data_sample['id']).drop_duplicates()
    
    idx = indices[id]
    
    #Aggregate rating added with cosine score in sim_score list
    
    simScore = []
    
    for i,j in enumerate(cosine_sim[idx]):
        k = data_sample['vendor_rating'].iloc[i]
        
        if j!=0:
            simScore.append((i,j,k))
    
    #Sort the restaurant names based on the similarity scores
    simScore = sorted(simScore, key = lambda x: (x[1],x[2]), reverse=True)
    
    # Top 10 similar hotels
    simScore = simScore[0:10]
    
    vendorIndices = [i[0] for i in simScore]
    
    top_10 = data_sample[['id','vendor_rating']].iloc[vendorIndices]
    
    top_10['similarity_score'] = 0
    
    for i,j in enumerate(simScore):
        top_10['similarity_score'].iloc[i] = round(simScore[i][1],2)
    
    return top_10
    
    
    
    
    

In [6]:
recommend_func(new_data, 13)

/Users/pw/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/pw/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/pw/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html

,id,vendor_rating,similarity_score
1,13,4.7,1.00
13,78,4.4,0.75
40,188,4.6,0.69
75,391,4.2,0.49
73,356,4.2,0.41
8,55,4.5,0.34
10,67,4.3,0.34
38,176,4.3,0.34
20,86,4.5,0.33
16,82,4.4,0.32
